
**Miembros del grupo:**

* Diego Alonso Herrera Ramírez, CC 70908268
* Sharid Samantha Madrid Ospina, CC 1001652997


**Base de datos:**
* [Heart Attack Analysis & Prediction Dataset](https://www.kaggle.com/datasets/rashikrahmanpritom/heart-attack-analysis-prediction-dataset/data): El objetivo es determinar las probabilidades de que un paciente tenga un ataque cardíaco en función de varias características médicas y de salud.

In [ ]:
pip install mlxtend

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

In [ ]:
# Carga el conjunto de datos original
df = pd.read_csv("https://raw.githubusercontent.com/Udeaproject/Heart-Attack-Analysis-Prediction/main/heart.csv?token=GHSAT0AAAAAACGKRVK4NV5IPU4XBGDWBIJAZJ24OJQ", delimiter=",")
df

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [ ]:
# Comprobación de duplicados en el conjunto de datos
duplicate_count = df.duplicated().sum()
if duplicate_count > 0:
    # Eliminación de duplicados
    df.drop_duplicates(inplace=True)

In [ ]:
# Comprobación de valores faltantes
missing_values = df.isnull().sum().sum()
if missing_values == 0:
    print("No hay valores faltantes")

No hay valores faltantes


In [ ]:
# Descripción estadística de los datos
df.describe().T


,count,mean,std,min,25%,50%,75%,max
age,302.0,54.420530,9.047970,29.0,48.00,55.5,61.00,77.0
sex,302.0,0.682119,0.466426,0.0,0.00,1.0,1.00,1.0
cp,302.0,0.963576,1.032044,0.0,0.00,1.0,2.00,3.0
trtbps,302.0,131.602649,17.563394,94.0,120.00,130.0,140.00,200.0
chol,302.0,246.500000,51.753489,126.0,211.00,240.5,274.75,564.0
fbs,302.0,0.149007,0.356686,0.0,0.00,0.0,0.00,1.0
restecg,302.0,0.526490,0.526027,0.0,0.00,1.0,1.00,2.0
thalachh,302.0,149.569536,22.903527,71.0,133.25,152.5,166.00,202.0
exng,302.0,0.327815,0.470196,0.0,0.00,0.0,1.00,1.0
oldpeak,302.0,1.043046,1.161452,0.0,0.00,0.8,1.60,6.2


In [ ]:
#Combiamos el nombre de algunas variables para que sea mas entendible

df.rename(columns={
    'cp': 'chest_pain_type',
    'chol':'cholestoral',
    'trtbps':'resting_bp',
    'fbs':'fasting_blood_sugar',
    'restecg':'rest_ecg',
    'thalachh':'max_heart_rate',
    'exng':'exercise_induced_angina',
    'caa':'no_major_vessels',
    'thall':'Thallium_stress_test',
    'output':"had_ht"
} , inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 302 entries, 0 to 302
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   age                      302 non-null    int64  
 1   sex                      302 non-null    int64  
 2   chest_pain_type          302 non-null    int64  
 3   resting_bp               302 non-null    int64  
 4   cholestoral              302 non-null    int64  
 5   fasting_blood_sugar      302 non-null    int64  
 6   rest_ecg                 302 non-null    int64  
 7   max_heart_rate           302 non-null    int64  
 8   exercise_induced_angina  302 non-null    int64  
 9   oldpeak                  302 non-null    float64
 10  slp                      302 non-null    int64  
 11  no_major_vessels         302 non-null    int64  
 12  Thallium_stress_test     302 non-null    int64  
 13  had_ht                   302 non-null    int64  
dtypes: float64(1), int64(13)
m

In [ ]:
df

,age,sex,chest_pain_type,resting_bp,cholestoral,fasting_blood_sugar,rest_ecg,max_heart_rate,exercise_induced_angina,oldpeak,slp,no_major_vessels,Thallium_stress_test,had_ht
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


#Procesamiento de Datos


In [ ]:
# Dividimos el Dataset en datos de entrenamiento y prueba

X = df.drop("had_ht", axis=1)
y = df['had_ht']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Estandarización de datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Balanceo de clases

In [ ]:
# Verificar la proporción de clases en el conjunto de entrenamiento
print(y_train.value_counts(normalize=True))

# Verificar la proporción de clases en el conjunto de prueba
print(y_test.value_counts(normalize=True))


1    0.547718
0    0.452282
Name: had_ht, dtype: float64
1    0.52459
0    0.47541
Name: had_ht, dtype: float64


### Rendimiento inicial del modelo

In [ ]:
# Entrenar tu modelo y evaluar su rendimiento
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train_scaled, y_train)

# Predicciones en el conjunto de prueba
y_pred = model.predict(X_test_scaled)

# Calcular la precisión u otra métrica de desempeño
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


Accuracy: 0.8360655737704918


In [ ]:
from sklearn.model_selection import cross_val_score, StratifiedKFold

# Usar validación cruzada
k_fold = StratifiedKFold(n_splits=8, shuffle=True, random_state=42)
cross_val_scores = cross_val_score(model, X_train_scaled, y_train, cv=k_fold, scoring='accuracy')

# Ver los puntajes de validación cruzada
print("Cross Validation Scores:", cross_val_scores)
print("Mean CV Accuracy:", cross_val_scores.mean())


Cross Validation Scores: [0.80645161 0.76666667 0.86666667 0.8        0.8        0.93333333
 0.83333333 0.86666667]
Mean CV Accuracy: 0.8341397849462366


##Modelo de Funciones Discriminantes Gaussianas (Gaussian Discriminant Analysis - GDA):

In [ ]:
# Crear el modelo GDA
gda_model = GaussianNB()

# Entrenar el modelo con los datos de entrenamiento
gda_model.fit(X_train_scaled, y_train)

# Predecir con los datos de prueba
gda_predictions = gda_model.predict(X_test_scaled)

# Calcular la precisión del modelo GDA
gda_accuracy = accuracy_score(y_test, gda_predictions)
print(f"Accuracy of Gaussian Discriminant Analysis: {gda_accuracy}")


Accuracy of Gaussian Discriminant Analysis: 0.8852459016393442


In [ ]:
# Modelo GDA
print("Gaussian Discriminant Analysis (GDA) Metrics:")
# Calcular el rendimiento mediante validación cruzada
k_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
cross_val_scores = cross_val_score(gda_model, X_train_scaled, y_train, cv=k_fold, scoring='accuracy')

# Imprimir resultados de validación cruzada
print(f"\nResultados de Validación Cruzada (Accuracy): {cross_val_scores}")
print(f"Accuracy Promedio: {cross_val_scores.mean():.4f}")
print(f"Desviación Estándar: {cross_val_scores.std():.4f}")

# Entrenar el modelo
gda_model.fit(X_train_scaled, y_train)  # Asegúrate de utilizar el conjunto de características seleccionadas

# Realizar predicciones en el conjunto de prueba
gda_predictions = gda_model.predict(X_test_scaled)
gda_confusion_matrix = confusion_matrix(y_test, gda_predictions)
gda_classification_report = classification_report(y_test, gda_predictions)

print("\nConfusion Matrix:")
print(gda_confusion_matrix)
print("\nClassification Report:")
print(gda_classification_report)


Gaussian Discriminant Analysis (GDA) Metrics:

Resultados de Validación Cruzada (Accuracy): [0.68       0.70833333 0.83333333 0.875      0.91666667 0.70833333
 0.79166667 0.75       0.91666667 0.875     ]
Accuracy Promedio: 0.8055
Desviación Estándar: 0.0855

Confusion Matrix:
[[27  2]
 [ 5 27]]

Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.93      0.89        29
           1       0.93      0.84      0.89        32

    accuracy                           0.89        61
   macro avg       0.89      0.89      0.89        61
weighted avg       0.89      0.89      0.89        61



##Modelo de K Vecinos Más Cercanos (K-Nearest Neighbors - KNN):

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
# Entrenar el modelo con 5  vecinos
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train_scaled, y_train)

# Predecir en el conjunto de prueba
knn_predictions = knn_model.predict(X_test_scaled)


# Evaluar el modelo

accuracy = accuracy_score(y_test, knn_predictions)
print(f"Accuracy of K-Nearest Neighbors (k=5): {accuracy}")



Accuracy of K-Nearest Neighbors (k=5): 0.9180327868852459


In [ ]:
# Modelo KNN
print("\nK-Nearest Neighbors (KNN) Metrics:")
# Validación cruzada de k-fold (k=5)
k_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Calcular el rendimiento mediante validación cruzada
cross_val_scores = cross_val_score(knn_model, X_train_scaled, y_train, cv=k_fold, scoring='accuracy')

knn_model.fit(X_train_scaled, y_train)


# Imprimir resultados de validación cruzada

print(f"\nResultados de Validación Cruzada (Accuracy): {cross_val_scores}")
print(f"\nAccuracy Promedio: {cross_val_scores.mean():.4f}")
print(f"\nDesviación Estándar: {cross_val_scores.std():.4f}")

knn_y_pred = knn_model.predict(X_test_scaled)
knn_confusion_matrix = confusion_matrix(y_test, knn_y_pred)
knn_classification_report = classification_report(y_test, knn_y_pred)

print("\nConfusion Matrix:")
print(knn_confusion_matrix)
print("\nClassification Report:")
print(knn_classification_report)


K-Nearest Neighbors (KNN) Metrics:

Resultados de Validación Cruzada (Accuracy): [0.7755102  0.8125     0.79166667 0.8125     0.85416667]

Accuracy Promedio: 0.8093

Desviación Estándar: 0.0264

Confusion Matrix:
[[28  1]
 [ 4 28]]

Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.97      0.92        29
           1       0.97      0.88      0.92        32

    accuracy                           0.92        61
   macro avg       0.92      0.92      0.92        61
weighted avg       0.92      0.92      0.92        61



##Modelo de Redes Neuronales Feed-Forward (Feed-Forward Neural Networks - FNN):

In [ ]:
# Creamos un objeto StandardScaler
scaler = StandardScaler()

# Ajustamos y transformamos los datos de entrenamiento
X_train_scaled = scaler.fit_transform(X_train)

# Transformamos los datos de prueba utilizando el mismo scaler
X_test_scaled = scaler.transform(X_test)

# Definimos el modelo con dos capas ocultas, activación Tanh y función de costo logística
fnn_model = MLPClassifier(hidden_layer_sizes=(50,100), activation='tanh', max_iter=10000)

# Entrenamos el modelo
fnn_model.fit(X_train_scaled, y_train)

# Evaluamos el modelo
fnn_accuracy = fnn_model.score(X_test_scaled, y_test)
print(f"Accuracy of Feed-Forward Neural Networks: {fnn_accuracy}")


Accuracy of Feed-Forward Neural Networks: 0.8852459016393442


In [ ]:
# Modelo FNN
print("\nFeed-Forward Neural Networks (FNN) Metrics:")
# Validación cruzada de k-fold (k=5)
k_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Calcular el rendimiento mediante validación cruzada
cross_val_scores = cross_val_score(fnn_model, X_train_scaled, y_train, cv=k_fold, scoring='accuracy')

# Imprimir resultados de validación cruzada

print(f"\nResultados de Validación Cruzada (Accuracy): {cross_val_scores}")
print(f"Accuracy Promedio: {cross_val_scores.mean():.4f}")
print(f"Desviación Estándar: {cross_val_scores.std():.4f}")

fnn_y_pred = fnn_model.predict(X_test_scaled)
fnn_confusion_matrix = confusion_matrix(y_test, fnn_y_pred)
fnn_classification_report = classification_report(y_test, fnn_y_pred)


print("Confusion Matrix:")
print(fnn_confusion_matrix)
print("\nClassification Report:")
print(fnn_classification_report)


Feed-Forward Neural Networks (FNN) Metrics:

Resultados de Validación Cruzada (Accuracy): [0.73469388 0.79166667 0.75       0.77083333 0.83333333]
Accuracy Promedio: 0.7761
Desviación Estándar: 0.0345
Confusion Matrix:
[[27  2]
 [ 5 27]]

Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.93      0.89        29
           1       0.93      0.84      0.89        32

    accuracy                           0.89        61
   macro avg       0.89      0.89      0.89        61
weighted avg       0.89      0.89      0.89        61



##Modelo de Bosque Aleatorio (Random Forest):

In [ ]:
# Ajustar hiperparámetros y manejar desequilibrio de clases
random_forest_model = RandomForestClassifier(n_estimators=100)
random_forest_model.fit(X_train_scaled, y_train)

# Evaluar el modelo
rf_accuracy = random_forest_model.score(X_test_scaled, y_test)

# Realizar validación cruzada
cv_scores = cross_val_score(random_forest_model, X_train_scaled, y_train, cv=5)
print(f"Cross-validated Accuracy of Random Forest: {np.mean(cv_scores)} +/- {np.std(cv_scores)}")

Cross-validated Accuracy of Random Forest: 0.8340986394557823 +/- 0.031813859957711174


In [ ]:
# Modelo Random Forest
print("\nRandom Forest Metrics:")
# Validación cruzada de k-fold (k=5)
k_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Calcular el rendimiento mediante validación cruzada
cross_val_scores = cross_val_score(random_forest_model, X_train_scaled, y_train, cv=k_fold, scoring='accuracy')

# Imprimir resultados de validación cruzada

print(f"\nResultados de Validación Cruzada (Accuracy): {cross_val_scores}")
print(f"Accuracy Promedio: {cross_val_scores.mean():.4f}")
print(f"Desviación Estándar: {cross_val_scores.std():.4f}")


rf_y_pred = random_forest_model.predict(X_test_scaled)
rf_confusion_matrix = confusion_matrix(y_test, rf_y_pred)
rf_classification_report = classification_report(y_test, rf_y_pred)


print("Confusion Matrix:")
print(rf_confusion_matrix)
print("\nClassification Report:")
print(rf_classification_report)


Random Forest Metrics:

Resultados de Validación Cruzada (Accuracy): [0.71428571 0.83333333 0.8125     0.70833333 0.85416667]
Accuracy Promedio: 0.7845
Desviación Estándar: 0.0612
Confusion Matrix:
[[26  3]
 [ 4 28]]

Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.90      0.88        29
           1       0.90      0.88      0.89        32

    accuracy                           0.89        61
   macro avg       0.88      0.89      0.89        61
weighted avg       0.89      0.89      0.89        61



##Modelo de Máquinas de Vectores de Soporte (Support Vector Machines - SVM):

In [ ]:
from sklearn.svm import SVC

C = 1
gamma = 0.1
kernel = 'poly'

# Inicializar el modelo SVM con los parámetros
svm_model = SVC(decision_function_shape='ovr',C=C, gamma=gamma, kernel=kernel)

svm_model.fit(X_train_scaled, y_train)

# Evaluar el modelo
svm_accuracy = svm_model.score(X_test_scaled, y_test)
print(f"Accuracy of Support Vector Machines: {svm_accuracy}")



Accuracy of Support Vector Machines: 0.8688524590163934


In [ ]:
# Modelo SVM
print("\nSupport Vector Machines (SVM) Metrics:")
# Validación cruzada de k-fold (k=5)
k_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Calcular el rendimiento mediante validación cruzada
cross_val_scores = cross_val_score(svm_model, X_train_scaled, y_train, cv=k_fold, scoring='accuracy')

# Imprimir resultados de validación cruzada

print(f"\nResultados de Validación Cruzada (Accuracy): {cross_val_scores}")
print(f"Accuracy Promedio: {cross_val_scores.mean():.4f}")
print(f"Desviación Estándar: {cross_val_scores.std():.4f}")


svm_y_pred = svm_model.predict(X_test_scaled)
svm_confusion_matrix = confusion_matrix(y_test, svm_y_pred)
svm_classification_report = classification_report(y_test, svm_y_pred)


print("\nConfusion Matrix:")
print(svm_confusion_matrix)
print("\nClassification Report:")
print(svm_classification_report)



Support Vector Machines (SVM) Metrics:

Resultados de Validación Cruzada (Accuracy): [0.73469388 0.70833333 0.83333333 0.75       0.85416667]
Accuracy Promedio: 0.7761
Desviación Estándar: 0.0572

Confusion Matrix:
[[26  3]
 [ 5 27]]

Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.90      0.87        29
           1       0.90      0.84      0.87        32

    accuracy                           0.87        61
   macro avg       0.87      0.87      0.87        61
weighted avg       0.87      0.87      0.87        61



#Selección de características

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
import time
import pandas as pd

In [ ]:
# Definir nombres y descripciones de las características (simulado)
feature_names = ['age', 'sex', 'cp', 'trtbps', 'chol', 'fbs', 'restecg', 'thalachh', 'exng', 'oldpeak']
feature_descriptions = ['Edad del paciente', 'Género del paciente', 'Tipo de dolor en el pecho', 'Presión arterial en reposo',
                        'Nivel de colesterol', 'Nivel de azúcar en sangre en ayunas', 'Resultados electrocardiográficos en reposo',
                        'Frecuencia cardíaca máxima alcanzada', 'Angina inducida por ejercicio', 'Depresión del segmento ST inducida por el ejercicio']

# Diccionario para almacenar características seleccionadas por cada método
selected_features = {}

In [ ]:
models = {'KNN': knn_model}

# Diccionario para almacenar resultados
results = {model: {} for model in models}
selected_features_info = {model: {} for model in models}
# Realizar selección de características para cada modelo
for model_name, model in models.items():
    for method in ['SFS', 'SBS', 'SFFS', 'SBFS']:
        sfs = SFS(model,
                  k_features=5,  # Número de características a seleccionar
                  forward=True if 'FS' in method else False,
                  floating=True if 'FFS' in method else False,
                  scoring='accuracy',
                  cv=5)

        sfs = sfs.fit(X_train, y_train)

        selected_features = X_train.columns[list(sfs.k_feature_idx_)]
        X_train_selected = X_train[selected_features]
        X_test_selected = X_test[selected_features]

        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)

        acc = accuracy_score(y_test, y_pred)

        results[model_name][method] = acc
        selected_features_info[model_name][method] = selected_features
# Crear dataframe con los resultados
results_df = pd.DataFrame(results)
print(results_df)

# Imprimir características seleccionadas por método y modelo
for model_name, info in selected_features_info.items():
    print(f"\nModelo: {model_name}")
    for method, features in info.items():
        print(f"\nMétodo: {method}")
        print(f"Características seleccionadas: {', '.join(features)}")

           KNN
SBFS  0.770492
SBS   0.852459
SFFS  0.770492
SFS   0.770492

Modelo: KNN

Método: SFS
Características seleccionadas: sex, rest_ecg, exercise_induced_angina, oldpeak, Thallium_stress_test

Método: SBS
Características seleccionadas: sex, oldpeak, slp, no_major_vessels, Thallium_stress_test

Método: SFFS
Características seleccionadas: sex, rest_ecg, exercise_induced_angina, oldpeak, Thallium_stress_test

Método: SBFS
Características seleccionadas: sex, rest_ecg, exercise_induced_angina, oldpeak, Thallium_stress_test


In [ ]:
models = {'FNN': fnn_model}

# Diccionario para almacenar resultados
results = {model: {} for model in models}
selected_features_info = {model: {} for model in models}
# Realizar selección de características para cada modelo
for model_name, model in models.items():
    for method in ['SFS', 'SBS', 'SFFS', 'SBFS']:
        sfs = SFS(model,
                  k_features=5,  # Número de características a seleccionar
                  forward=True if 'FS' in method else False,
                  floating=True if 'FFS' in method else False,
                  scoring='accuracy',
                  cv=5)

        sfs = sfs.fit(X_train, y_train)

        selected_features = X_train.columns[list(sfs.k_feature_idx_)]
        X_train_selected = X_train[selected_features]
        X_test_selected = X_test[selected_features]

        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)

        acc = accuracy_score(y_test, y_pred)

        results[model_name][method] = acc
        selected_features_info[model_name][method] = selected_features
# Crear dataframe con los resultados
results_df = pd.DataFrame(results)
print(results_df)

# Imprimir características seleccionadas por método y modelo
for model_name, info in selected_features_info.items():
    print(f"\nModelo: {model_name}")
    for method, features in info.items():
        print(f"\nMétodo: {method}")
        print(f"Características seleccionadas: {', '.join(features)}")

In [ ]:
models = { 'SVM': svm_model}

# Diccionario para almacenar resultados
results = {model: {} for model in models}
selected_features_info = {model: {} for model in models}
# Realizar selección de características para cada modelo
for model_name, model in models.items():
    for method in ['SFS', 'SBS', 'SFFS', 'SBFS']:
        sfs = SFS(model,
                  k_features=5,  # Número de características a seleccionar
                  forward=True if 'FS' in method else False,
                  floating=True if 'FFS' in method else False,
                  scoring='accuracy',
                  cv=5)

        sfs = sfs.fit(X_train, y_train)

        selected_features = X_train.columns[list(sfs.k_feature_idx_)]
        X_train_selected = X_train[selected_features]
        X_test_selected = X_test[selected_features]

        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)

        acc = accuracy_score(y_test, y_pred)

        results[model_name][method] = acc
        selected_features_info[model_name][method] = selected_features
# Crear dataframe con los resultados
results_df = pd.DataFrame(results)
print(results_df)

# Imprimir características seleccionadas por método y modelo
for model_name, info in selected_features_info.items():
    print(f"\nModelo: {model_name}")
    for method, features in info.items():
        print(f"\nMétodo: {method}")
        print(f"Características seleccionadas: {', '.join(features)}")